In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
from utils import *

In [2]:
df_tennis=pd.read_csv('../data/prepared_data.csv')
df_tennis.head()

,player_id,player_name,player_hand,player_ht,player_age,ace,df,svpt,1stIn,1stWon,...,SvGms,bpSaved,bpFaced,rank,opponent_rank,surface,tourney_level,best_of,target,month
0,100644,Alexander Zverev,R,198,26,10,1,66,47,33,...,10,3,4,7,5,Hard,A,3,1,11
1,100644,Alexander Zverev,R,198,25,5,1,44,27,16,...,8,3,7,15,97,Hard,M,3,0,3
2,100644,Alexander Zverev,R,198,26,5,5,90,66,50,...,15,3,6,19,81,Clay,A,3,1,7
3,100644,Alexander Zverev,R,198,26,3,1,41,33,26,...,7,2,2,19,122,Clay,A,3,1,7
4,100644,Alexander Zverev,R,198,26,7,0,55,42,20,...,8,2,6,10,50,Hard,M,3,0,10


In [3]:
df_tennis.columns

Index(['player_id', 'player_name', 'player_hand', 'player_ht', 'player_age',
       'ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved',
       'bpFaced', 'rank', 'opponent_rank', 'surface', 'tourney_level',
       'best_of', 'target', 'month'],
      dtype='object')

In [4]:
df_tennis.drop('player_id',axis=1,inplace=True)
df_tennis.head()

,player_name,player_hand,player_ht,player_age,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,rank,opponent_rank,surface,tourney_level,best_of,target,month
0,Alexander Zverev,R,198,26,10,1,66,47,33,11,10,3,4,7,5,Hard,A,3,1,11
1,Alexander Zverev,R,198,25,5,1,44,27,16,7,8,3,7,15,97,Hard,M,3,0,3
2,Alexander Zverev,R,198,26,5,5,90,66,50,8,15,3,6,19,81,Clay,A,3,1,7
3,Alexander Zverev,R,198,26,3,1,41,33,26,4,7,2,2,19,122,Clay,A,3,1,7
4,Alexander Zverev,R,198,26,7,0,55,42,20,8,8,2,6,10,50,Hard,M,3,0,10


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2023-12-12 03:52:09.154737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
#label encoding
le=LabelEncoder()
df_tennis['player_hand_encoded']=le.fit_transform(df_tennis['player_hand'])

#hot encoding
df_tennis=pd.get_dummies(df_tennis, columns=['surface', 'tourney_level'], drop_first=True)

#standardization numeric features
scaler=StandardScaler()
numerical_cols=['player_ht', 'player_age', 'ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved', 'bpFaced', 'rank', 'opponent_rank']

df_tennis[numerical_cols]=scaler.fit_transform(df_tennis[numerical_cols])

#splitting data
X=df_tennis.drop(['target', 'player_name', 'player_hand'], axis=1)
y=df_tennis['target']

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)





In [7]:
#building model
model=Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) #sigmoid because we have binary classification

#compiling model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
#training model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
110/110 [==============================] - 1s 4ms/step - loss: 0.6103 - accuracy: 0.6728 - val_loss: 0.5459 - val_accuracy: 0.7398
Epoch 2/10
110/110 [==============================] - 0s 2ms/step - loss: 0.5039 - accuracy: 0.7547 - val_loss: 0.4893 - val_accuracy: 0.7648
Epoch 3/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4587 - accuracy: 0.7794 - val_loss: 0.4829 - val_accuracy: 0.7659
Epoch 4/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4330 - accuracy: 0.7905 - val_loss: 0.4569 - val_accuracy: 0.7818
Epoch 5/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4190 - accuracy: 0.7951 - val_loss: 0.4414 - val_accuracy: 0.7943
Epoch 6/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4067 - accuracy: 0.8073 - val_loss: 0.4342 - val_accuracy: 0.7932
Epoch 7/10
110/110 [==============================] - 0s 2ms/step - loss: 0.3979 - accuracy: 0.8104 - val_loss: 0.4391 - val_accuracy: 0.7932
Epoch 

In [9]:
#evaluating model
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5).astype(int).reshape(X_test.shape[0])

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Classification Report: \n', classification_report(y_test, y_pred))

35/35 [==============================] - 0s 2ms/step
Accuracy: 0.75
Confusion Matrix: 
 [[474  77]
 [197 352]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.71      0.86      0.78       551
           1       0.82      0.64      0.72       549

    accuracy                           0.75      1100
   macro avg       0.76      0.75      0.75      1100
weighted avg       0.76      0.75      0.75      1100



<h2>Let's hypertuning gridsearch </h2>

In [11]:
from tensorflow.keras.layers import Dropout

def create_model(optimizer='adam', activation='relu', neurons=64, dropout_rate=0.0):
    model=Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [12]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [13]:
model=KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)


/var/folders/pn/lq_2s97j34j4yncm576jk9_r0000gn/T/ipykernel_55949/1310235282.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)


In [14]:
param_Grid={
    'optimizer':['adam', 'sgd'],
    'dropout_rate':[0.0, 0.1, 0.2],
}

grid_search=GridSearchCV(estimator=model, param_grid=param_Grid, scoring='precision', cv=3)

#fit the model
grid_result=grid_search.fit(X_train, y_train)

print('Best parameters: ', grid_result.best_params_)

46/46 [==============================] - 0s 1ms/step
Best parameters:  {'dropout_rate': 0.2, 'optimizer': 'adam'}


In [15]:
def create_tuned_model(dropout_rate=0.2, optimizer='adam'):
    model=Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [16]:
tuned_model=create_tuned_model()

tuned_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred=tuned_model.predict(X_test)
y_pred=(y_pred>0.5).astype(int).reshape(X_test.shape[0])

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Classification Report: \n', classification_report(y_test, y_pred))

Epoch 1/10
110/110 [==============================] - 2s 5ms/step - loss: 0.6863 - accuracy: 0.5816 - val_loss: 0.5727 - val_accuracy: 0.7170
Epoch 2/10
110/110 [==============================] - 0s 2ms/step - loss: 0.5757 - accuracy: 0.6978 - val_loss: 0.5196 - val_accuracy: 0.7455
Epoch 3/10
110/110 [==============================] - 0s 2ms/step - loss: 0.5340 - accuracy: 0.7243 - val_loss: 0.4950 - val_accuracy: 0.7818
Epoch 4/10
110/110 [==============================] - 0s 2ms/step - loss: 0.5135 - accuracy: 0.7425 - val_loss: 0.4906 - val_accuracy: 0.7705
Epoch 5/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4826 - accuracy: 0.7604 - val_loss: 0.4619 - val_accuracy: 0.7898
Epoch 6/10
110/110 [==============================] - 0s 4ms/step - loss: 0.4774 - accuracy: 0.7621 - val_loss: 0.4507 - val_accuracy: 0.7977
Epoch 7/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4526 - accuracy: 0.7803 - val_loss: 0.4432 - val_accuracy: 0.7943
Epoch 

<h2>Let's hypertuning learning rate </h2>

In [19]:
from tensorflow.keras.optimizers import Adam, SGD
def create_model_learning_rate( dropout_rate=0.2, learning_rate=0.001):
    optimizer=Adam(learning_rate=learning_rate)
    model=Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [20]:
model=KerasClassifier(build_fn=create_model_learning_rate, epochs=10, batch_size=32, verbose=0)

param_grid={
    'learning_rate':[0.001, 0.01, 0.1],
}

grid_search=GridSearchCV(estimator=model, param_grid=param_grid, scoring='precision', cv=3)

grid_result=grid_search.fit(X_train, y_train)

print('Best parameters: ', grid_result.best_params_)

/var/folders/pn/lq_2s97j34j4yncm576jk9_r0000gn/T/ipykernel_55949/3057274832.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model_learning_rate, epochs=10, batch_size=32, verbose=0)


46/46 [==============================] - 0s 1ms/step


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best parameters:  {'learning_rate': 0.01}


In [21]:
model_tuned=create_model_learning_rate(learning_rate=0.01)

In [22]:
model_tuned.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred=model_tuned.predict(X_test)
y_pred=(y_pred>0.5).astype(int).reshape(X_test.shape[0])

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Classification Report: \n', classification_report(y_test, y_pred))

Epoch 1/10
110/110 [==============================] - 2s 5ms/step - loss: 0.5804 - accuracy: 0.6944 - val_loss: 0.5072 - val_accuracy: 0.7591
Epoch 2/10
110/110 [==============================] - 0s 3ms/step - loss: 0.4793 - accuracy: 0.7629 - val_loss: 0.4898 - val_accuracy: 0.7580
Epoch 3/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4453 - accuracy: 0.7814 - val_loss: 0.4369 - val_accuracy: 0.8011
Epoch 4/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4369 - accuracy: 0.7897 - val_loss: 0.4305 - val_accuracy: 0.7920
Epoch 5/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4305 - accuracy: 0.7979 - val_loss: 0.4307 - val_accuracy: 0.7875
Epoch 6/10
110/110 [==============================] - 0s 3ms/step - loss: 0.4249 - accuracy: 0.7931 - val_loss: 0.5005 - val_accuracy: 0.7602
Epoch 7/10
110/110 [==============================] - 0s 3ms/step - loss: 0.4123 - accuracy: 0.8059 - val_loss: 0.4248 - val_accuracy: 0.7886
Epoch 

<h2>Let's hypertuning model architecture </h2>

In [23]:
def create_model_architecture(neurons_layer=64, neurons_layer2=32):
    model=Sequential()
    model.add(Dense(neurons_layer, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(neurons_layer2, activation='relu'))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [24]:
model=KerasClassifier(build_fn=create_model_architecture, epochs=10, batch_size=32, verbose=0)

param_grid={
    'neurons_layer':[32, 64, 128],
    'neurons_layer2':[16, 32, 64]
}

grid_search=GridSearchCV(estimator=model, param_grid=param_grid, scoring='precision', cv=3)
grid_result=grid_search.fit(X_train, y_train)

print('Best parameters: ', grid_result.best_params_)

/var/folders/pn/lq_2s97j34j4yncm576jk9_r0000gn/T/ipykernel_55949/3078346017.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model_architecture, epochs=10, batch_size=32, verbose=0)


46/46 [==============================] - 0s 1ms/step
Best parameters:  {'neurons_layer': 128, 'neurons_layer2': 16}


In [25]:
def create_optimized_model(dropout_rate=0.2, neurons_layer=128, neurons_layer2=16, learning_rate=0.01):
    optimizer=Adam(learning_rate=learning_rate)
    model=Sequential()
    model.add(Dense(neurons_layer, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons_layer2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [26]:
optimized_model=create_optimized_model()

optimized_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred=optimized_model.predict(X_test)
y_pred=(y_pred>0.5).astype(int).reshape(X_test.shape[0])

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Classification Report: \n', classification_report(y_test, y_pred))

Epoch 1/10
110/110 [==============================] - 2s 4ms/step - loss: 0.5552 - accuracy: 0.7146 - val_loss: 0.5023 - val_accuracy: 0.7773
Epoch 2/10
110/110 [==============================] - 0s 4ms/step - loss: 0.4789 - accuracy: 0.7683 - val_loss: 0.4597 - val_accuracy: 0.7807
Epoch 3/10
110/110 [==============================] - 0s 3ms/step - loss: 0.4561 - accuracy: 0.7797 - val_loss: 0.4450 - val_accuracy: 0.7966
Epoch 4/10
110/110 [==============================] - 0s 3ms/step - loss: 0.4413 - accuracy: 0.7897 - val_loss: 0.4677 - val_accuracy: 0.7886
Epoch 5/10
110/110 [==============================] - 0s 4ms/step - loss: 0.4245 - accuracy: 0.7959 - val_loss: 0.4295 - val_accuracy: 0.7966
Epoch 6/10
110/110 [==============================] - 0s 4ms/step - loss: 0.4202 - accuracy: 0.7990 - val_loss: 0.4314 - val_accuracy: 0.7977
Epoch 7/10
110/110 [==============================] - 1s 5ms/step - loss: 0.4164 - accuracy: 0.7973 - val_loss: 0.4472 - val_accuracy: 0.7784
Epoch 

In [27]:
threshold=0.5
test_set_probability_of_wining=optimized_model.predict(X_test)
test_set_prediction=(test_set_probability_of_wining>threshold).astype(int).reshape(X_test.shape[0])

35/35 [==============================] - 0s 2ms/step
